In [1]:
!pip install fastapi uvicorn pyngrok streamlit nest_asyncio sqlalchemy passlib    --quiet

In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("34pCLI9eTKpGu5XZQjzoYHCHUBD_2Eprorppjy7T3TDToP3wi")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# backend.py
%%writefile backend.py
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import sqlite3
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# FASTAPI INITIALIZATION
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all during development
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

DB_PATH = "/content/drive/MyDrive/students.db"

# DATABASE HELPER FUNCTION


def db_connection():
    return sqlite3.connect(DB_PATH)

# REQUEST MODELS

class LoginRequest(BaseModel):
    student_id: int
    password: str
    role: str

class SignupRequest(BaseModel):
    student_id: int
    name: str
    email: str
    username: str
    password: str
    role: str

class StudyLog(BaseModel):
    student_id: int
    date: str
    study_hour: float
    subject: str
    method_used: str
    distraction_time: float
    quiz_score: float

class StudyLogUpdate(BaseModel):
    date: str
    study_hour: float
    subject: str
    method_used: str
    distraction_time: float
    quiz_score: float

# LOGIN API

@app.post("/login")
def login(data: LoginRequest):
    conn = db_connection()
    cursor = conn.cursor()

    cursor.execute("""
        SELECT * FROM students
        WHERE student_id=? AND password=? AND role=?
    """, (data.student_id, data.password, data.role))

    row = cursor.fetchone()
    conn.close()

    if row:
        student = {
            "student_id": row[0],
            "name": row[1],
            "email": row[2],
            "username": row[3],
            "password": row[4],
            "role": row[5]
        }
        return {"status": "success", "student": student}
    else:
        raise HTTPException(status_code=401, detail="Invalid credentials")

# SIGNUP / ADD STUDENT
@app.post("/signup")
def signup(data: SignupRequest):
    conn = db_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("""
            INSERT INTO students (student_id, name, email, username, password, role)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (data.student_id, data.name, data.email, data.username, data.password, data.role))

        conn.commit()
        return {"status": "success", "message": "Student added successfully"}

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

    finally:
        conn.close()

# ADD STUDY LOG

from datetime import datetime

@app.post("/add_study_log")
def add_study_log(log: StudyLog):
    try:
        # convert date to day_name

        from datetime import datetime, date

        date_str = log.date  # this is a string

        # Try both date formats
        try:
            dt = datetime.strptime(date_str, "%m/%d/%Y")
        except:
            dt = datetime.strptime(date_str, "%Y-%m-%d")

        formatted = dt.strftime("%m/%d/%Y")

        day_name = dt.strftime("%A")     # Monday, Tuesday, etc.

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO study_logs (student_id, date, day_name, study_hour, subject, method_used, distraction_time, quiz_score)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            log.student_id,
            log.date,
            day_name,
            log.study_hour,
            log.subject,
            log.method_used,
            log.distraction_time,
            log.quiz_score
        ))

        conn.commit()
        conn.close()

        return {"message": "Study log added successfully!"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.put("/edit_log/{log_id}")
def edit_log(log_id: int, log: StudyLogUpdate):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # validate date format
        try:
            try:
                dt = datetime.strptime(log.date, "%m/%d/%Y")
            except:
                dt = datetime.strptime(log.date, "%Y-%m-%d")
            day_name = dt.strftime("%A")
        except:
            raise HTTPException(status_code=400, detail="Invalid date format")

        cursor.execute("""
            UPDATE study_logs
            SET date=?, day_name=?, study_hour=?, subject=?, method_used=?, distraction_time=?, quiz_score=?
            WHERE log_id=?
        """, (
            log.date,
            day_name,
            log.study_hour,
            log.subject,
            log.method_used,
            log.distraction_time,
            log.quiz_score,
            log_id
        ))

        conn.commit()
        conn.close()

        if cursor.rowcount == 0:
            raise HTTPException(status_code=404, detail="Log ID not found")

        return {"message": "Log updated successfully"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.delete("/delete_log/{log_id}")
def delete_log(log_id: int):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        cursor.execute("DELETE FROM study_logs WHERE log_id=?", (log_id,))
        conn.commit()
        conn.close()

        if cursor.rowcount == 0:
            raise HTTPException(status_code=404, detail="Log ID not found")

        return {"message": "Log deleted successfully"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# GET STUDY LOGS
@app.get("/study_logs/{student_id}")
def get_logs(student_id: int):
    conn = db_connection()
    cursor = conn.cursor()

    cursor.execute("""
        SELECT * FROM study_logs WHERE student_id=?
    """, (student_id,))
    rows = cursor.fetchall()
    conn.close()

    logs = []
    for r in rows:
        logs.append({
            "log_id": r[0],
            "student_id": r[1],
            "date": r[2],
            "day_name": r[3],
            "study_hour": r[4],
            "subject": r[5],
            "method_used": r[6],
            "distraction_time": r[7],
            "quiz_score": r[8]
        })

    return {"study_logs": logs}

# VIEW ALL STUDY LOGS (ADMIN)

@app.get("/all_study_logs")
def all_logs():
    conn = db_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM study_logs")
    rows = cursor.fetchall()
    conn.close()

    logs = []
    for r in rows:
        logs.append({
            "log_id": r[0],
            "student_id": r[1],
            "date": r[2],
            "day_name": r[3],
            "study_hour": r[4],
            "subject": r[5],
            "method_used": r[6],
            "distraction_time": r[7],
            "quiz_score": r[8]
        })

    return {"all_study_logs": logs}




@app.get("/recommend/{student_id}")
def recommend(student_id: int):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.cluster import KMeans

    try:
       # FETCH STUDY LOGS
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("""
            SELECT date, study_hour, subject, method_used, distraction_time, quiz_score
            FROM study_logs
            WHERE student_id=?
        """, (student_id,))
        rows = cursor.fetchall()
        conn.close()

        if not rows:
            return {"message": "No study logs available yet.", "recommendations": {}}

        df = pd.DataFrame(rows, columns=[
            "date", "study_hour", "subject", "method_used",
            "distraction_time", "quiz_score"
        ])

        #  ITEM-BASED COLLABORATIVE FILTERING (CF)
        subject_scores = df.groupby("subject")["quiz_score"].mean()

        if len(subject_scores) <= 1:
            recommended_subject = subject_scores.idxmax()
        else:
            pivot = df.pivot_table(values="quiz_score", index="subject", aggfunc="mean")
            similarity = pivot.corr()
            last_subject = df.iloc[-1]["subject"]

            if last_subject in similarity.columns:
                sim_scores = similarity[last_subject].dropna()
                recommended_subject = sim_scores.nlargest(2).index[-1] \
                    if len(sim_scores) > 1 else last_subject
            else:
                recommended_subject = subject_scores.idxmax()

        # TOP SUBJECT (Strongest Performance)

        top_subject = subject_scores.idxmax()

        #  WEAK SUBJECT (Lowest Quiz Score)
        weak_subject = subject_scores.idxmin()

        # K-MEANS CLUSTERING (4 clusters)

        df_ml = df[["study_hour", "quiz_score", "distraction_time"]].copy()
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df_ml)

        kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')
        cluster_label = kmeans.fit_predict(df_scaled)[-1]  # student’s latest cluster

        cluster_mean_quiz = df["quiz_score"].mean()

        #  METHOD RECOMMENDATION (choose method with best performance)
        method_scores = df.groupby("method_used")["quiz_score"].mean()
        recommended_method = method_scores.idxmax()

        #  ONLINE/OFFLINE TOOL MAPPING

        tool_map = {
            "Videos": "YouTube, Khan Academy",
            "Flashcards": "Quizlet, Anki",
            "Notes": "Notion, OneNote",
            "Group Study": "Google Meet, Discord Study Room",
            "Reading": "NCERT Online, PDFs",
        }
        recommended_tool = tool_map.get(recommended_method, "General Study Tools")

        #  ADVICE ENGINE (Rule-Based)

        advice_list = []

        if df["distraction_time"].mean() > 25:
            advice_list.append("Reduce mobile usage—try Forest App.")
        if df["study_hour"].mean() < 2:
            advice_list.append("Increase study hours gradually to at least 2–3 hrs/day.")
        if df["quiz_score"].mean() < 60:
            advice_list.append("Revise weak topics daily for 20 minutes.")

        if not advice_list:
            advice_list.append("You are doing great! Keep consistency.")

        #  MOTIVATION MESSAGE

        motivation = np.random.choice([
            "You are improving every day — keep going!",
            "Small steps lead to big success.",
            "Believe in yourself — you are capable!",
            "Stay consistent, progress is happening!",
            "Your hard work will pay off!"
        ])

        #  FINAL RESPONSE
        result = {
            "Recommended Subject (CF)": recommended_subject,
            "Top Subject": top_subject,
            "Weak Subject": weak_subject,
            "Recommended Study Method": recommended_method,
            "Recommended Tool": recommended_tool,
            "Cluster Category": int(cluster_label),
            "Advice": advice_list,
            "Motivation": motivation,
        }

        return result

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



Overwriting backend.py


In [5]:
import nest_asyncio, uvicorn, threading
from pyngrok import ngrok

nest_asyncio.apply()
ngrok.kill()

public_url = ngrok.connect(8000)
print(" FastAPI Public URL:", public_url.public_url)

def run_backend():
    uvicorn.run("backend:app", host="0.0.0.0", port=8000)

threading.Thread(target=run_backend).start()

 FastAPI Public URL: https://cognately-evocative-cristin.ngrok-free.dev


In [6]:
!pkill -f ngrok
!pkill -f uvicorn

In [7]:
!mkdir -p .streamlit

In [8]:
%%writefile .streamlit/config.toml
[theme]
base="light"
primaryColor="#E63946"
backgroundColor="#F1FAEE"
secondaryBackgroundColor="#A8DADC"
textColor="#1D3557"


Overwriting .streamlit/config.toml


In [9]:
!ls -a .streamlit


.  ..  config.toml


In [10]:
%%writefile frontend.py

import streamlit as st
import requests
import pandas as pd

FASTAPI_URL = "http://localhost:8000"

st.set_page_config(page_title=" Study System", layout="wide")

# --- SESSION MANAGEMENT ---
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.student = None
if "page" not in st.session_state:
    st.session_state.page = "dashboard"
if "selected_student" not in st.session_state:
    st.session_state.selected_student = None

# --- LOGIN PAGE ---
def login_page():
    st.title("Study Habit Recommender")
    student_id = st.text_input("Student ID")
    password = st.text_input("Password", type="password")
    role = st.selectbox("Role", ["Student", "admin"])
    if st.button("Login"):
        if not student_id or not password:
            st.warning("Enter both ID and Password.")
        else:
            try:
                response = requests.post(f"{FASTAPI_URL}/login",
                                         json={"student_id": int(student_id), "password": password, "role": role})
                if response.status_code == 200:
                    data = response.json()
                    st.session_state.logged_in = True
                    st.session_state.student = data["student"]
                    st.session_state.page = "dashboard"
                    st.success(f"Welcome, {data['student']['name']}!")
                    st.stop()
                else:
                    st.error(" Invalid credentials")
            except Exception as e:
                st.error(f"Cannot connect to backend: {e}")

# --- SIGNUP PAGE ---
def signup_page():
    st.title("Register New Student")
    student_id = st.text_input("Student ID")
    name = st.text_input("Full Name")
    email = st.text_input("Email")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    role = st.selectbox("Role", ["Student", "admin"])
    if st.button("Signup"):
        if not all([student_id, name, email, username, password]):
            st.warning("Fill all fields.")
        else:
            try:
                response = requests.post(f"{FASTAPI_URL}/signup",
                                         json={"student_id": int(student_id), "name": name, "email": email,
                                               "username": username, "password": password, "role": role})
                if response.status_code == 200:
                    st.success(" Registered successfully!")
                else:
                    st.error(" Error during signup")
            except Exception as e:
                st.error(f"Cannot connect to backend: {e}")

# --- STUDENT DASHBOARD ---
def student_dashboard():
    student = st.session_state.student
    st.sidebar.title(" Navigation")
    st.sidebar.write(f"{student['name']} ({student['role']})")

    if st.sidebar.button("Logout"):
        st.session_state.logged_in = False
        st.session_state.student = None
        st.session_state.page = "dashboard"
        st.stop()

    st.title(f"Welcome, {student['name']}!")
    st.subheader(" Student Dashboard")

    col1, col2 = st.columns(2)
    with col1:
        if st.button(" Add Study Log"):
            st.session_state.page = "add_log"
            st.stop()
    with col2:
        if st.button(" Get Recommendation"):
            st.session_state.page = "recommendation"
            st.stop()

    # Fetch Study Logs First

    logs = []

    try:
        resp = requests.get(f"{FASTAPI_URL}/study_logs/{student['student_id']}")
        if resp.status_code == 200:
            logs = resp.json().get("study_logs", [])
        else:
            st.error("Error fetching study logs")
    except Exception as e:
        st.error(f" Cannot fetch study logs: {e}")

    # Show Logs
       # Show Logs
    if logs:
        df = pd.DataFrame(logs).rename(columns={
            "log_id": "Log ID",
            "date": "Date",
            "day_name": "Day",
            "study_hour": "Study Hours",
            "subject": "Subject",
            "method_used": "Method Used",
            "distraction_time": "Distraction Time",
            "quiz_score": "Quiz Score"
        })
        st.dataframe(df)
    else:
        st.info("No study logs found.")


    # VISUALIZATION SECTION (after logs loaded)

    if logs:
        st.subheader(" Study Insights")

        df = pd.DataFrame(logs)

        # Convert date column
        try:
            df["date"] = pd.to_datetime(df["date"])
        except:
            pass

        #  Line Chart – Study Hours Over Time
        st.line_chart(df[["date", "study_hour"]].set_index("date"))

        #  Bar Chart – Study Hours by Subject
        st.bar_chart(df.groupby("subject")["study_hour"].sum())

        # Pie Chart – Subject Time Distribution
        pie_df = df.groupby("subject")["study_hour"].sum()
        st.pyplot(pie_df.plot.pie(autopct="%1.1f%%").figure)

        #  Quiz Score Trend
        st.subheader(" Quiz Score Trend")
        st.line_chart(df[["subject", "quiz_score"]].set_index("subject"))


# --- ADMIN DASHBOARD ---
def admin_dashboard():
    st.sidebar.title(" Navigation")
    st.sidebar.write(f" {st.session_state.student['name']} (Admin)")
    if st.sidebar.button("Logout"):
        st.session_state.logged_in = False
        st.session_state.student = None
        st.session_state.page = "dashboard"
        st.stop()

    st.subheader("Admin Dashboard")
    col1, col2, col3 = st.columns(3)
    with col1:
        if st.button(" Add Student"):
            st.session_state.page = "add_student"
            st.stop()
    with col2:
        if st.button(" Edit Student"):
            st.session_state.page = "edit_logs"
            st.stop()
    with col3:
        if st.button(" View Students"):
            st.session_state.page = "view_students"
            st.stop()


# EDIT LOG REQUEST

def edit_logs_page():
    st.header("Edit / Delete Study Logs")

    # Fetch all logs for admin
    try:
        resp = requests.get(f"{FASTAPI_URL}/all_study_logs")
        if resp.status_code != 200:
            st.error("Error fetching logs")
            return
        logs = resp.json()["all_study_logs"]
    except Exception as e:
        st.error(f"Backend error: {e}")
        return

    if not logs:
        st.info("No logs available to edit.")
        return

    df = pd.DataFrame(logs)
    st.dataframe(df)

    # Select a log
    log_ids = df["log_id"].tolist()
    chosen = st.selectbox("Select Log ID to Edit/Delete", log_ids)

    selected_row = df[df["log_id"] == chosen].iloc[0]

    st.subheader("Edit Selected Log")

    date = st.text_input("Date", selected_row["date"])
    study_hour = st.number_input("Study Hours", value=float(selected_row["study_hour"]))
    subject = st.text_input("Subject", selected_row["subject"])
    method_used = st.text_input("Method Used", selected_row["method_used"])
    distraction_time = st.number_input("Distraction Time", value=float(selected_row["distraction_time"]))
    quiz_score = st.number_input("Quiz Score", value=float(selected_row["quiz_score"]))

    col1, col2 = st.columns(2)

    with col1:
        if st.button(" Update Log"):
            payload = {
                "date": date,
                "study_hour": study_hour,
                "subject": subject,
                "method_used": method_used,
                "distraction_time": distraction_time,
                "quiz_score": quiz_score
            }
            r = requests.put(f"{FASTAPI_URL}/edit_log/{chosen}", json=payload)
            if r.status_code == 200:
                st.success("Log updated successfully!")
                st.experimental_rerun()
            else:
                st.error(f"Update failed: {r.text}")

    with col2:
        if st.button("🗑 Delete Log"):
            r = requests.delete(f"{FASTAPI_URL}/delete_log/{chosen}")
            if r.status_code == 200:
                st.success("Log deleted!")
                st.experimental_rerun()
            else:
                st.error(f"Delete failed: {r.text}")

    #  Back button (correct indentation)
    if st.button(" Back to Dashboard"):
        st.session_state.page = "dashboard"
        st.rerun()



# --- ADD STUDY LOG ---
def add_log_page():
    st.header(" Add Study Log")

    student = st.session_state.student
    student_id = student["student_id"]

    with st.form("log_form"):
        date = st.date_input("Date")
        study_hour = st.number_input("Study Hours", min_value=0.0)
        subject = st.selectbox("Subject", ["Biology", "English", "Computer", "Math", "Science", "History"])
        method_used = st.selectbox("Study Method", ['Pomodoro', 'Notes Review', 'Flashcards', 'Group Study', 'Practice Test', 'Videos',
                                                    'Active recall', 'Mind Map','Teaching a Friend','Past Papers'])
        distraction_time = st.number_input("Distraction Time (minutes)", min_value=0.0)
        quiz_score = st.number_input("Quiz Score", min_value=0.0)

        submitted = st.form_submit_button("Add Log")

        if submitted:
            log_data = {
                "student_id": student_id,
                "date": str(date),
                "study_hour": study_hour,
                "subject": subject,
                "method_used": method_used,
                "distraction_time": distraction_time,
                "quiz_score": quiz_score
            }

            try:
                resp = requests.post(f"{FASTAPI_URL}/add_study_log", json=log_data)
                if resp.status_code == 200:
                    st.success(" Study Log Added Successfully!")
                else:
                    st.error(f" Error: {resp.text}")
            except Exception as e:
                st.error(f" Cannot connect to backend: {e}")

    if st.button(" Back to Dashboard"):
        st.session_state.page = "dashboard"
        st.stop()


# --- ADD STUDENT PAGE ---
def add_student_page():
    st.header(" Add New Student")
    with st.form("add_student_form"):
        student_id = st.number_input("Student ID", min_value=1)
        name = st.text_input("Full Name")
        email = st.text_input("Email")
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")
        role = st.selectbox("Role", ["Student", "admin"])
        submitted = st.form_submit_button("Add Student")
        if submitted:
            try:
                resp = requests.post(f"{FASTAPI_URL}/signup",
                                     json={"student_id": int(student_id), "name": name, "email": email,
                                           "username": username, "password": password, "role": role})
                if resp.status_code == 200:
                    st.success(" Student added successfully!")
                else:
                    st.error(" Error adding student")
            except Exception as e:
                st.error(f" Cannot connect to backend: {e}")
    if st.button(" Back to Dashboard"):
        st.session_state.page = "dashboard"
        st.stop()

# --- VIEW STUDENTS PAGE ---
def view_all_logs_page():
    st.header(" All Students Study Logs")


    # Load logs from backend only

    try:
        resp = requests.get(f"{FASTAPI_URL}/all_study_logs")
        if resp.status_code != 200:
            st.error("Error fetching study logs")
            return
        logs = resp.json().get("all_study_logs", [])
    except Exception as e:
        st.error(f" Cannot fetch study logs: {e}")
        return

    if not logs:
        st.info("No study logs found.")
        return

    df = pd.DataFrame(logs)
    st.dataframe(df)


    # Analytics
    st.subheader(" Analytics Dashboard")

    df["date"] = pd.to_datetime(df["date"], errors="ignore")

    colA, colB = st.columns(2)

    with colA:
        st.write(" Total Study Hours per Subject")
        st.bar_chart(df.groupby("subject")["study_hour"].sum())

    with colB:
        st.write(" Average Quiz Score per Subject")
        st.bar_chart(df.groupby("subject")["quiz_score"].mean())

    # Daily hours line chart
    st.write(" Daily Total Study Hours")
    daily_hours = df.groupby("date")["study_hour"].sum()
    st.line_chart(daily_hours)

    # Box Plot
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    df.boxplot(column="study_hour", by="subject", ax=ax)
    st.pyplot(fig)

    if st.button(" Back to Dashboard"):
        st.session_state.page = "dashboard"
        st.stop()



def recommendation_page():
    st.title("Personalized AI Study Recommendations")

    student = st.session_state.student
    student_id = student["student_id"]

    st.info(f"Fetching recommendations for **{student['name']}** (ID: {student_id})...")

    try:
        resp = requests.get(f"{FASTAPI_URL}/recommend/{student_id}")
        if resp.status_code != 200:
            st.error("Error fetching recommendations")
            return

        rec = resp.json()

        # ----- CARDS -----
        st.subheader(" Your AI Recommendations")

        col1, col2 = st.columns(2)

        with col1:
            st.markdown(f"""
            <div style="padding:15px; border-radius:12px; background:#e3f2fd;">
                <h4> Recommended Subject (CF)</h4>
                <p style="font-size:20px; color:#0d47a1;"><b>{rec['Recommended Subject (CF)']}</b></p>
            </div>
            """, unsafe_allow_html=True)

        with col2:
            st.markdown(f"""
            <div style="padding:15px; border-radius:12px; background:#e8f5e9;">
                <h4> Top Performing Subject</h4>
                <p style="font-size:20px; color:#1b5e20;"><b>{rec['Top Subject']}</b></p>
            </div>
            """, unsafe_allow_html=True)

        col3, col4 = st.columns(2)

        with col3:
            st.markdown(f"""
            <div style="padding:15px; border-radius:12px; background:#ffebee;">
                <h4> Weak Subject</h4>
                <p style="font-size:20px; color:#b71c1c;"><b>{rec['Weak Subject']}</b></p>
            </div>
            """, unsafe_allow_html=True)

        with col4:
            st.markdown(f"""
            <div style="padding:15px; border-radius:12px; background:#fff3e0;">
                <h4> Best Study Method</h4>
                <p style="font-size:20px; color:#e65100;"><b>{rec['Recommended Study Method']}</b></p>
            </div>
            """, unsafe_allow_html=True)

        # ----- TOOL RECOMMENDATION -----
        st.markdown(f"""
        <div style="padding:15px; border-radius:12px; background:#ede7f6; margin-top:20px;">
            <h4> Recommended Tool</h4>
            <p style="font-size:20px; color:#4a148c;"><b>{rec['Recommended Tool']}</b></p>
        </div>
        """, unsafe_allow_html=True)

        # ----- CLUSTER CATEGORY -----
        cluster_meanings = {
            0: " Cluster 0 – Low scores, high distractions → Focus improvement needed",
            1: " Cluster 1 – Moderate performance → Keep consistency",
            2: " Cluster 2 – High performance → Excellent work!",
            3: " Cluster 3 – Good potential but inconsistent"
        }

        cluster_text = cluster_meanings.get(rec["Cluster Category"], "Unknown Cluster")

        st.markdown(f"""
        <div style="padding:15px; border-radius:12px; background:#f1f8e9; margin-top:20px;">
            <h4> Performance Category (K-Means)</h4>
            <p style="font-size:18px;"><b>{cluster_text}</b></p>
        </div>
        """, unsafe_allow_html=True)

        # ----- ADVICE -----
        st.subheader(" Personalized Advice")
        for advice in rec["Advice"]:
            st.markdown(f" {advice}")

        # ----- MOTIVATION -----
        st.markdown("<br>", unsafe_allow_html=True)

        st.markdown(f"""
        <div style="padding:20px; border-radius:12px; background:#e0f7fa; text-align:center;">
            <h3> Motivation</h3>
            <p style="font-size:22px;"><b>{rec['Motivation']}</b></p>
        </div>
        """, unsafe_allow_html=True)

        # ---- PERFORMANCE VISUALIZATION ----
        st.subheader(" Your Performance Graphs")

        try:
            logs_resp = requests.get(f"{FASTAPI_URL}/study_logs/{student_id}")

            if logs_resp.status_code == 200:
                logs_data = logs_resp.json().get("study_logs", [])
                df = pd.DataFrame(logs_data)

                if len(df) > 0:
                    df["date"] = pd.to_datetime(df["date"])

                    col1, col2 = st.columns(2)

                    with col1:
                        st.write(" Study Hours Over Time")
                        st.line_chart(df[["date", "study_hour"]].set_index("date"))

                    with col2:
                        st.write(" Quiz Scores Over Time")
                        st.line_chart(df[["date", "quiz_score"]].set_index("date"))

                    st.write(" Best vs Weak Subjects (Bar Graph)")
                    st.bar_chart(df.groupby("subject")["quiz_score"].mean())

        except Exception as e:
            st.error(f"Error loading graphs: {e}")

    except Exception as e:
        st.error(f"Error fetching recommendation: {e}")

    if st.button(" Back to Dashboard"):
        st.session_state.page = "dashboard"
        st.stop()

def analytics_page():
    st.header(" Analytics Dashboard")

    # --- Fetch logs from backend ---
    try:
        resp = requests.get(f"{FASTAPI_URL}/all_study_logs")
        logs = resp.json()["all_study_logs"]
    except:
        st.error("Failed to load analytics data")
        return

    if len(logs) == 0:
        st.warning("No study logs available")
        return

    import pandas as pd

    df = pd.DataFrame(logs)

    st.subheader(" Total Study Hours per Subject")
    if "study_hour" in df.columns:
        sub_hours = df.groupby("subject")["study_hour"].sum()
        st.bar_chart(sub_hours)

    st.subheader(" Average Quiz Score per Subject")
    if "quiz_score" in df.columns:
        sub_quiz = df.groupby("subject")["quiz_score"].mean()
        st.bar_chart(sub_quiz)

    st.subheader(" Daily Total Study Hours")
    try:
        df["date"] = pd.to_datetime(df["date"])
        daily = df.groupby("date")["study_hour"].sum()
        st.line_chart(daily)
    except:
        st.warning("Date format issue in logs")


# --- MAIN ROUTER ---
if not st.session_state.logged_in:
    menu = st.sidebar.radio("Select Option", ["Login", "Signup"])
    if menu == "Login":
        login_page()
    else:
        signup_page()

else:
    page = st.session_state.page
    role = st.session_state.student["role"].lower()

    if role == "admin":
        if page == "dashboard":
            admin_dashboard()
        elif page == "add_student":
            add_student_page()
        elif page == "view_students":
            view_all_logs_page()
        elif page == "edit_logs":
            edit_logs_page()
        elif page == "analytics":
            analytics_page()
        else:
            st.info("Page under construction")

    else:  # student
        if page == "dashboard":
            student_dashboard()
        elif page == "add_log":
            add_log_page()
        elif st.session_state.page == "recommendation":
            recommendation_page()
        else:
            student_dashboard()



Overwriting frontend.py


In [11]:
from pyngrok import ngrok
!streamlit run frontend.py &>/content/logs.txt &
public_url = ngrok.connect(8501)
print("Streamlit Public URL:", public_url)

Streamlit Public URL: NgrokTunnel: "https://cognately-evocative-cristin.ngrok-free.dev" -> "http://localhost:8501"
